In [1]:
import pystac_client
import pystac
import odc.stac
import rioxarray
import pathlib
import pandas
import geopandas
import numpy
import folium
import dotenv
import branca.element, branca.colormap # Remove whitespace around small folium map

# Across Otago
* Look at breaking into smaller sections
* Discuss how to break up the country
* Where to compute - Jupyter Lab Compernucus vs HPC
* Displays - Area covered by date, Spectrum plots, designated water + kelp areas + spectrum

In [2]:
def geopandas_bounds_to_plot(dataframe, crs=4326):
    """ Changing bounding box representation to leaflet notation ``(lon1, lat1, lon2, lat2) -> ((lat1, lon1), (lat2, lon2))`` """
    x1, y1, x2, y2 = dataframe.to_crs(crs).total_bounds
    return ((y1, x1), (y2, x2))

In [3]:
def check_offset_scale(search):
    """ Check expected offset and scale """
    for item in search.item_collection():
        for band in bands:
            if band == "scl": 
                continue
            if item.assets[band].extra_fields['raster:bands'][0]['offset'] != OFFSET:
                raise Exception("offset doesn't match expected")
            if item.assets[band].extra_fields['raster:bands'][0]['scale'] != SCALE:
                raise Exception("scale doesn't match expected")

In [4]:
data_path = pathlib.Path.cwd() / ".." / "data"
crs_wsg = 4326
crs = 2193
name = "waikouaiti"
(data_path / "rasters" / name).mkdir(parents=True, exist_ok=True)
(data_path / "vectors").mkdir(parents=True, exist_ok=True)

## Setup input datasets

In [5]:
if not (data_path / "vectors" / "regions.gpkg").exists() or not (data_path / "vectors" / "main_islands.gpkg").exists():
    dotenv.load_dotenv()
    linz_key = os.environ.get("LINZ_API", None)
    fetcher = geoapis.vector.Linz(linz_key, verbose=False, crs=crs)
    regions = fetcher.run(50785)
    islands = fetcher.run(51153)
    
    main_islands = islands[islands.area > 9e8]
    
    regions.to_file(data_path / "vectors" / "regions.gpkg")
    main_islands.to_file(data_path / "vectors" / "main_islands.gpkg")

NameError: name 'os' is not defined

## Create Kelp raster maps

In [ ]:
bands = ["red", "green", "blue", "nir", "scl"]
filters = {"eo:cloud_cover":{"lt":10}} 
SCL_CIRRUS = 10
SCL_CLOUD = 9
SCL_DEFECTIVE = 1
SCL_NO_DATA = 0
OFFSET = -0.1
SCALE = 0.0001
thresholds = {"min_ndvi": 0.01, "max_ndvi": 0.7, "max_ndwi": 0.2}

In [ ]:
# use publically available stac link such as
odc.stac.configure_rio(cloud_defaults=True, aws={"aws_unsigned": True})
client = pystac_client.Client.open("https://earth-search.aws.element84.com/v1") 

# ID of the collection
collection = "sentinel-2-c1-l2a"

# Geometry of AOI - convex hull to allow search
land = geopandas.read_file(data_path / "vectors" / "main_islands.geojson")
geometry_df = geopandas.read_file(data_path / "vectors" / f"{name.lower()}.gpkg")
geometry_query = geometry_df .to_crs(crs_wsg).iloc[0].geometry

In [ ]:
year = 2020
months = [f"{year}-{str(month).zfill(2)}" for month in list(range(1, 13))]

In [ ]:
for month_YYMM in months:
    print(f"Check month: {month_YYMM}")
    # run pystac client search to see available dataset
    search = client.search(
        collections=[collection], intersects=geometry_query, datetime=month_YYMM, query=filters
    ) 

    if len(search.item_collection()) == 0:
        continue # Nothing meeting criteria for this month

    check_offset_scale(search)

    data = odc.stac.load(search.items(), geopolygon=geometry_query, bands=bands,  chunks={}, groupby="solar_day")

    # remove if no data
    data["scl"].load()
    data["scl"] = data["scl"].rio.clip(land.to_crs(data["scl"].rio.crs).geometry.values, invert=True, drop=False)
    data["scl"].rio.write_crs(data["scl"].rio.crs, inplace=True);
    data = data.isel(time=(data["scl"] != SCL_NO_DATA).any(dim=["x", "y"]))

    # convert bands to actual values
    for band in bands: 
        if band == "scl": 
            continue
        data[band].data = data[band].data * SCALE + OFFSET
        data[band].data[data[band].data <= 0] = numpy.nan
        data[band].rio.write_nodata(numpy.nan, encoded=True, inplace=True);

    # Calculate NVDI and NVWI
    data["ndvi"] = (data.nir - data.red) / (data.nir + data.red)
    data["ndwi"] = (data.green-data.nir)/(data.green+data.nir)
    data["ndvi"].rio.write_crs(data["ndvi"].rio.crs, inplace=True); data["ndvi"].rio.write_nodata(numpy.nan, encoded=True, inplace=True);
    data["ndwi"].rio.write_crs(data["ndvi"].rio.crs, inplace=True); data["ndwi"].rio.write_nodata(numpy.nan, encoded=True, inplace=True); 

    # Calculate Kelp
    data["kelp"] = (data.nir - data.red) / (data.nir + data.red)
    data["kelp"].data[(data["ndvi"].data < thresholds["min_ndvi"]) | (data["ndvi"].data > thresholds["max_ndvi"]) | (data["ndwi"].data > thresholds["max_ndwi"])] = numpy.nan
    data["kelp"].data[(data["scl"].data == SCL_CIRRUS) | (data["ndvi"].data == SCL_CLOUD) | (data["ndwi"].data == SCL_DEFECTIVE)] = numpy.nan
    data["kelp"] = data["kelp"].rio.clip(land.to_crs(data["kelp"].rio.crs).geometry.values, invert=True)
    data["kelp"].rio.write_crs(data["kelp"].rio.crs, inplace=True); data["kelp"].rio.write_nodata(numpy.nan, encoded=True, inplace=True);

    # Save each separately
    for index in range(len(data["kelp"].time)):
        data["kelp"].isel(time=index).rio.to_raster(data_path / "rasters" / name / f'kelp_{pandas.to_datetime(data["kelp"].time.data[index]).strftime("%Y-%m-%d")}.tif', compress="deflate")

In [6]:
files = list(pathlib.Path(data_path / "rasters" / name).glob(f"*.tif"))

In [7]:
index = 0
kelp_display = rioxarray.rioxarray.open_rasterio(files[index], chunks=True).squeeze( "band", drop=True)

In [8]:
fig = branca.element.Figure(width='70%', height='50%') # Ensures no extra whitespace below

m = folium.Map()
land.explore(m=m)

kelp_display.odc.add_to(m, opacity=0.75, cmap="inferno", vmin=0, vmax=1) # viridis
# Bounds - either: geopandas_bounds_to_plot(land), or data.kelp.odc.map_bounds()
m.fit_bounds(kelp_display.odc.map_bounds())

colormap = branca.colormap.linear.inferno.scale(0, 1)
colormap.caption = 'Kelp Index'
colormap.add_to(m)

display(m)

NameError: name 'land' is not defined